# Word2GM Training Data Pipeline

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare a Google 5gram corpora for Word2GM skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` NVMe storage
2. **Processing**: TensorFlow-native filtering, vocabulary building, and triplet generation
3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [1]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_data_preprocessing_notebook, enable_autoreload

# Enable autoreload for development
enable_autoreload()

# Set up environment (CPU-only for data preprocessing)
env = setup_data_preprocessing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
batch_prepare_training_data = env['batch_prepare_training_data']
print_resource_summary = env['print_resource_summary']

<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: CPU-only</pre>

<pre>Data preprocessing environment ready!</pre>

## Print Resource Summary

In [2]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: cm008.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Requested partitions: short,cs,cm,cpu_a100_2,cpu_a100_1,cpu_gpu
   Running on: SSH failed: Host key verification failed.
   Job ID: 63372711
   Node list: cm008

GPU Information:
   Error: NVML Shared Library Not Found

TensorFlow GPU Detection:
   TensorFlow detects 0 GPU(s)
   Built with CUDA: True
============================================================</pre>

## Prepare Corpora

Here, we run the data-preparation pipeline from start to finish — reading preprocessed ngram corpora, generating all valid triplets, extracting the vocabulary, and saving the triplets and vocabulary as `tfrecord` files.

### Options for Data Preparation

You can control which years are processed and how the batch preparation runs by adjusting the arguments to `batch_prepare_training_data`:

**Ways to specify years:**
- `year_range="2010"` — Process a single year (e.g., only 2010).
- `year_range="2010,2012,2015"` — Process a comma-separated list of years.
- `year_range="2010-2015"` — Process a range of years, inclusive (2010 through 2015).
- `year_range="2010,2012-2014,2016"` — Combine individual years and ranges (2010, 2012, 2013, 2014, 2016).

**Other options:**
- `compress` — If `True`, output TFRecords are gzip-compressed. If `False`, output is uncompressed.
- `show_progress` — If `True`, display a progress bar for each year.
- `show_summary` — If `True`, print a summary of the processed data for each year.
- `use_multiprocessing` — If `True`, process years in parallel using multiple CPU cores (recommended for large datasets).

See the function docstring or source for more advanced options.

In [ ]:
# Configuration
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

downsample_threshold = 1e-5  # Typical values: 1e-3 (aggressive), 1e-4, 1e-5 (conservative)

# Process year(s)
results = batch_prepare_training_data(
    corpus_dir=corpus_dir,
    year_range="1700-1800",
    compress=False,
    show_progress=True,
    show_summary=True,
    use_multiprocessing=True,
    downsample_threshold=downsample_threshold
    # You can adjust downsample_threshold above to control how aggressively frequent words are downsampled.
    # Set to None or remove to disable downsampling.
    # See pipeline.py for details.
 )

Requested 101 years, found 101 corpus files.

PARALLEL BATCH PROCESSING
Processing 101 years
Using 14 parallel workers
Estimated speedup: 14.0x


1712 complete (1/101): 23 triplets, 36 vocab, 0.6s
1702 complete (2/101): 63 triplets, 75 vocab, 0.6s
1707 complete (3/101): 117 triplets, 108 vocab, 0.8s
1700 complete (4/101): 381 triplets, 325 vocab, 1.0s
1700 complete (4/101): 381 triplets, 325 vocab, 1.0s
1713 complete (5/101): 257 triplets, 210 vocab, 1.5s
1713 complete (5/101): 257 triplets, 210 vocab, 1.5s
1706 complete (6/101): 737 triplets, 483 vocab, 1.8s
1717 complete (7/101): 314 triplets, 312 vocab, 0.8s
1709 complete (8/101): 738 triplets, 465 vocab, 1.9s
1706 complete (6/101): 737 triplets, 483 vocab, 1.8s
1717 complete (7/101): 314 triplets, 312 vocab, 0.8s
1709 complete (8/101): 738 triplets, 465 vocab, 1.9s
1718 complete (9/101): 361 triplets, 321 vocab, 0.8s
1701 complete (10/101): 881 triplets, 542 vocab, 2.4s
1708 complete (11/101): 905 triplets, 564 vocab, 2.4s
1714 complete (12/101): 1,601 triplets, 834 vocab, 1.9s
1703 complete (13/101): 1,589 triplets, 794 vocab, 2.5s
1718 complete (9/101): 361 triplets, 321 v